## Captioning / Scene Description

In [ ]:
import cv2
import json
import numpy as np
import re

from datetime import timedelta
from os import listdir, makedirs, path

from PIL import Image as PImage

VIDEO_DB_PATH = "./metadata/keyframe-500/videos.json"

VIDEO_PATH = "../../vids/0801-500"
DIR_PATTERN = re.compile("^[0-3][0-9]-")

### Open Video Data

In [ ]:
with open(VIDEO_DB_PATH, "r") as f:
  video_data = json.load(f)

### Run Model

In [ ]:
input_dirs = sorted([d for d in listdir(VIDEO_PATH) if DIR_PATTERN.search(d) is not None])

for io_dir in input_dirs[:1]:
  input_dir_path = path.join(VIDEO_PATH, io_dir)
  input_files = sorted([f for f in listdir(input_dir_path) if f.endswith("mp4")])

  for io_file in input_files:
    if io_file not in video_data:
      print(io_file, "not in video_data")
      continue

    print(io_dir, io_file)
    input_file_path = path.join(input_dir_path, io_file)

    vid = cv2.VideoCapture(input_file_path)
    vw = int(vid.get(cv2.CAP_PROP_FRAME_WIDTH))
    vh = int(vid.get(cv2.CAP_PROP_FRAME_HEIGHT))
    frame_count = int(vid.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = int(vid.get(cv2.CAP_PROP_FPS))
    rep_frames = video_data[io_file]["representative_frames"]

    vid.set(cv2.CAP_PROP_POS_FRAMES, 0)
    for frame_data in rep_frames:
      frameIdx = frame_data["index"]
      vid.set(cv2.CAP_PROP_POS_FRAMES, frameIdx)
      _, frame = vid.read()
      # TODO: magic goes here
      frame_grey = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
      frame_data["caption"] = ""

    vid.release()

### Model tests

In [ ]:
input_dir = sorted([d for d in listdir(VIDEO_PATH) if DIR_PATTERN.search(d) is not None])[0]
input_dir_path = path.join(VIDEO_PATH, input_dir)
input_files = sorted([f for f in listdir(input_dir_path) if f.endswith("mp4")])
input_file_path = path.join(input_dir_path, input_files[3])

vid = cv2.VideoCapture(input_file_path)
vid.set(cv2.CAP_PROP_POS_FRAMES, 10)
_, frame = vid.read()

frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
image = PImage.fromarray(frame)

vid.release()

In [ ]:
from IPython.display import display, Image
display(image)

In [ ]:
from transformers import BlipProcessor, BlipForConditionalGeneration

MODELS = [
  "Salesforce/blip-image-captioning-base",
  "Salesforce/blip-image-captioning-large"
]

processor = BlipProcessor.from_pretrained(MODELS[1])
model = BlipForConditionalGeneration.from_pretrained(MODELS[1]).to("cuda")

inputs = processor(image, return_tensors="pt").to("cuda")

out = model.generate(**inputs)
caption = processor.decode(out[0], skip_special_tokens=True)
print(caption)

In [ ]:
from transformers import AutoProcessor, AutoModelForCausalLM

MODELS = [
  "microsoft/git-base",
  "microsoft/git-large",
]

processor = AutoProcessor.from_pretrained(MODELS[1])
model = AutoModelForCausalLM.from_pretrained(MODELS[1]).to("cuda")

pixel_values = processor(images=image, return_tensors="pt").pixel_values.to("cuda")

out = model.generate(pixel_values=pixel_values, max_length=50)
caption = processor.batch_decode(out, skip_special_tokens=True)[0]
print(caption)

In [ ]:
from transformers import VisionEncoderDecoderModel, ViTImageProcessor, AutoTokenizer

MODELS = [
  "nlpconnect/vit-gpt2-image-captioning"
]

model = VisionEncoderDecoderModel.from_pretrained(MODELS[0]).to("cuda")
processor = ViTImageProcessor.from_pretrained(MODELS[0])
tokenizer = AutoTokenizer.from_pretrained(MODELS[0])

pixel_values = processor(images=image, return_tensors="pt").pixel_values.to("cuda")

out = model.generate(pixel_values, max_length=48, num_beams=4)
caption = tokenizer.batch_decode(out, skip_special_tokens=True)[0]
print(caption)

### Time Tests

In [ ]:
input_dir = sorted([d for d in listdir(VIDEO_PATH) if DIR_PATTERN.search(d) is not None])[0]
input_dir_path = path.join(VIDEO_PATH, input_dir)
input_files = sorted([f for f in listdir(input_dir_path) if f.endswith("mp4")])

In [ ]:
%%time

for io_file in input_files:
  input_file_path = path.join(input_dir_path, io_file)
  vid = cv2.VideoCapture(input_file_path)
  rep_frames = video_data[io_file]["representative_frames"]

  vid.set(cv2.CAP_PROP_POS_FRAMES, 0)
  for frame_data in rep_frames:
    frameIdx = frame_data["index"]
    vid.set(cv2.CAP_PROP_POS_FRAMES, frameIdx)
    _, frame = vid.read()
    frame_grey = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
  vid.release()

In [ ]:
%%time

for io_file in input_files:
  input_file_path = path.join(input_dir_path, io_file)
  vid = cv2.VideoCapture(input_file_path)
  rep_frames = video_data[io_file]["representative_frames"]
  rep_frame_idxs = [f["index"] for f in rep_frames]

  vid.set(cv2.CAP_PROP_POS_FRAMES, 0)
  for frameIdx in range(0, frame_count):
    _, frame = vid.read()
    if frameIdx not in rep_frame_idxs:
      continue
    else:
      frame_grey = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
  vid.release()

### Post-Process (add to metadata)

In [ ]:
import json
import re

from os import listdir, path

VIDEO_DB_PATH = "./metadata/videos.json"
CAPTION_PATH = "./metadata/XXXX"

DIR_PATTERN = re.compile("^[0-3][0-9]-")

In [ ]:
# open all caption files
cap_data = {}

input_dirs = sorted([d for d in listdir(CAPTION_PATH) if DIR_PATTERN.search(d) is not None])

for io_dir in input_dirs:
  input_dir_path = path.join(CAPTION_PATH, io_dir)
  input_files = sorted([f for f in listdir(input_dir_path) if f.endswith("json")])

  for io_file in input_files:
    input_file_path = path.join(input_dir_path, io_file)
    video_key = io_file.replace("json", "mp4")
    with open(input_file_path, "r") as f:
      cap_data[video_key] = json.load(f)

In [ ]:
with open(VIDEO_DB_PATH, "r") as f:
  video_data = json.load(f)

In [ ]:
for k, vdata in video_data.items():
  if k not in cap_data:
    print(k, "has no caption info")
  else:
    for kk, kdata in cap_data[k].items():
      video_data[k] = vdata | kdata


In [ ]:
with open(VIDEO_DB_PATH, "w") as f:
  json.dump(video_data, f, indent=2, separators=(',',':'))